In [1]:
import numpy as np 
from random import random,uniform
import matplotlib.pyplot as plt 
from math import sqrt
%matplotlib qt

In [2]:
X_train = np.linspace(-5, 5, 2000)
y_train = -2 * X_train + 5 + np.random.rand() * 0.75
m = len(X_train)

def predict(theta0,theta1):
    h = theta0 * X_train + theta1 
    return h

In [3]:
class cost_function:
    def __init__(self) :
        self.x_range = np.array([-10,10],dtype=np.float32)
        self.y_range = np.array([-10,10],dtype=np.float32)
        self.minima = 0.0
        self.location = np.array([0,0],dtype=np.float32).reshape(1,2)

    def Query(self,theta0,theta1):
        h = theta0 * X_train + theta1 
        error = h - y_train
        return 1 / (2 * m) * sum(error**2)

In [4]:
class Bat_ALgorithm :
    def __init__(self,optimizer,Population_Size =100,Num_Movements = 100):
        # Functions informations
        self.x_range = optimizer.x_range 
        self.y_range = optimizer.y_range

        # Bat Population and Dynamics
        self.Population_Size = Population_Size
        self.Population , self.Cost = self.Initialize_Population(optimizer)

        self.Loudness = np.ones(shape=(self.Population_Size,1),dtype=np.float32)
        self.Pulse_Rate = np.zeros(shape=(self.Population_Size,1),dtype=np.float32)
        self.Frequency = np.zeros(shape=(self.Population_Size,2),dtype=np.float32)
        self.Velocity = np.ones(shape=(self.Population_Size,2),dtype=np.float32)

        # Create Internal algorithm paramaters
        self.Movements = Num_Movements
        self.Frequency_Range = np.array([0.0,1.4],dtype=np.float32)
        
        #arbetrary values can manipulate them to our needs 
        self.Loudness_Decay = 1.55
        self.Loudness_Limit = 0.1995
        self.Pulse_Rate_Decay = 1.5
        self.Gamma = 1.5

        # Optimization Results 
        self.Best_Position = np.zeros(shape=(1,2),dtype=np.float32)
        self.Best_Cost = 0.0
        self.Best_Bat = 0


    def Initialize_Population(self,optimizer):
        Population = np.zeros(shape=(self.Population_Size,2),dtype=np.float32)
        Cost = np.zeros(shape=(self.Population_Size,1),dtype=np.float32)

        for i in range (self.Population_Size):
            Population[i,0] = (self.x_range[1] - self.x_range[0]) * np.float32(random()) + self.x_range[0]
            Population[i,1] = (self.y_range[1] - self.y_range[0]) * np.float32(random()) + self.y_range[0]
            Cost[i,0] = optimizer.Query(Population[i,0],Population[i,1])
        
        return Population , Cost


    def Update_Dynamics (self,bat,best):
        self.Frequency[bat,:] = self.Frequency_Range[0] + (self.Frequency_Range[1] - self.Frequency_Range[0] * np.random.rand(1,self.Population.shape[1]))
        self.Velocity[bat,:] = (self.Population[best,:] - self.Population[bat,:]) * self.Frequency[bat,:]
        position = self.Population[bat,:] + self.Velocity[bat,:]

        return position   

    def Run (self,optimizer):
        best_fit = self.Cost.min()
        best_index = self.Cost.argmin()

        
        plt.figure('Solving regression problem using Bat optimization', figsize = (20,20))
        plt.subplot(1, 2, 1)
        Bats_plot = plt.scatter(self.Population[:, 0], self.Population[:, 1], c='b',marker='*', alpha=0.5, label = 'Bats')
        Best_plot = plt.scatter(self.Best_Position[:, 0], self.Best_Position[:, 1], c='r', marker='*', s=200, label = 'Best Position')
        plt.title('BAT OPTIMIZATION ',fontsize=25)
        plt.xticks([])
        plt.yticks([])
        plt.legend()
        
        
        plt.subplot(1, 2, 2)
        y_true = plt.plot(X_train, y_train, color = 'r', linewidth = 2, label = 'y_true')
        
        y_pred = predict(self.Best_Position[:, 0], self.Best_Position[:, 1])
        line = plt.plot(X_train, y_pred, color = 'b', linestyle = '--', linewidth = 2, label = 'y_predict')
        
        plt.title('Regression line', fontsize = 25)
        plt.xticks([])
        plt.yticks([])
        plt.legend()
        
        
        
        #Iterate through the simulation 
        for step in range (1,self.Movements+1):
            
            for bat in range (self.Population_Size):

                # update the position of the bat 
                position = self.Update_Dynamics(bat,best_index)

                # apply random walk 
                if random() < self.Loudness[bat,:] :
                    position+= self.Loudness.mean() * np.float32(uniform(-1,1))

                #check the limits of the search space 
                if position[0] < optimizer.x_range[0]:
                    position[0] = optimizer.x_range[0]
                elif position[0] > optimizer.x_range[1]:
                    position[0] = optimizer.x_range[1] 

                if position[1] < optimizer.y_range[0]:
                    position[1] = optimizer.y_range[0]
                elif position[1] > optimizer.y_range[1]:
                    position[1] = optimizer.y_range[1]               

                #check the Cost of the bat 
                Bat_Cost = optimizer.Query(position[0],position[1])

                if Bat_Cost < self.Cost[bat,:]:
                    self.Cost[bat,:] = Bat_Cost
                    self.Population[bat,:] = position
                    self.Pulse_Rate[bat,:] = self.Pulse_Rate_Decay * (1-np.exp(-self.Gamma * step))

                    if self.Loudness[bat,0] > self.Loudness_Limit :
                        self.Loudness[bat,0] *= self.Loudness_Decay
                    else :
                        self.Loudness[bat,0] *= self.Loudness_Limit

                    if Bat_Cost < best_fit :
                        best_fit = Bat_Cost
                        best_index = bat

                        #update best values 
                        self.Best_Position = position.reshape(1,2)
                        self.Best_Cost = Bat_Cost
                        self.Best_Bat = bat 

                plt.setp(Bats_plot, offsets = np.column_stack((self.Population[:, 0], self.Population[:, 1])))
                plt.setp(Best_plot, offsets = np.column_stack((self.Best_Position[:, 0], self.Best_Position[:, 1])))
                
                y_pred = predict(self.Best_Position[:, 0], self.Best_Position[:, 1])  
                plt.setp(line, linestyle='--', linewidth=2, ydata = y_pred)
                
                plt.pause(0.01)

In [5]:
cost_function = cost_function()
bat_algorithm = Bat_ALgorithm(cost_function, Population_Size=70, Num_Movements=100)
%time _ = bat_algorithm.Run(cost_function)
# Display the result from simulation 
print('='*50)
print('Optimizer')
print('='*50)
print(f'Best Cost {bat_algorithm.Best_Cost}')
print(f'Best Position : X: {bat_algorithm.Best_Position[0,0]}, Y: {bat_algorithm.Best_Position[0,1]} ')
print('='*50)
print('Function')
print('='*50)
print(f'Global Minimum {cost_function.minima}')
print(f'Location X: {cost_function.location[0,0]}, Y: {cost_function.location[0,1]}')
print('='*50)


